<a href="https://colab.research.google.com/github/reymillerc/Ethereum_Gas_Fee_App/blob/Weekly-Heatmap/Weekly_Heatmap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
import datetime
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


from google.colab import userdata
from IPython.display import clear_output  # Import clear_output function

# Retrieve the secret from Colab's secrets
INFURA_API_KEY = userdata.get('infurakey1')

# Function to fetch block information
def fetch_block(block_number):
    url = f'https://mainnet.infura.io/v3/{INFURA_API_KEY}'

    payload = {
        "jsonrpc": "2.0",
        "method": "eth_getBlockByNumber",
        "params": [block_number, True],
        "id": 1
    }
    headers = {'content-type': 'application/json'}
    response = requests.post(url, data=json.dumps(payload), headers=headers).json()
    return response

# Function to process transactions in a block and calculate average gas price
def process_blocks_and_calculate_average(start_block_number, end_block_number, step):
    hour_gas_prices = {}  # Dictionary to store gas prices for each hour

    # Iterate through blocks with the given step
    for block_number in range(start_block_number, end_block_number + 1, step):
        block_info = fetch_block(hex(block_number))
        timestamp = int(block_info.get('result', {}).get('timestamp', 0), 16)
        hour = timestamp // 3600  # Calculate hour based on timestamp

        if hour not in hour_gas_prices:
            hour_gas_prices[hour] = {'total_gas_price': 0, 'total_transactions': 0}

        transactions = block_info.get('result', {}).get('transactions', [])

        # Calculate gas price for each transaction in the block
        for tx in transactions:
            gas_price = int(tx.get('gasPrice', 0), 16)
            hour_gas_prices[hour]['total_gas_price'] += gas_price
            hour_gas_prices[hour]['total_transactions'] += 1

    # Calculate average gas price for each hour
    average_gas_prices = {}
    for hour, data in hour_gas_prices.items():
        if data['total_transactions'] > 0:
            average_gas_prices[hour] = data['total_gas_price'] / data['total_transactions'] / 10**9  # Convert to Gwei
        else:
            average_gas_prices[hour] = 0  # If no transactions found, set average gas price to 0

    return average_gas_prices

# Define the range of blocks to check (49800 blocks)
latest_block_number_hex = fetch_block("latest")['result']['number']
latest_block_number = int(latest_block_number_hex, 16)
start_block_number = latest_block_number - 49800 # Approx. number of blocks created in one week
end_block_number = latest_block_number

# Set the step size (fetch every 150th block)
step = 150


# Process blocks and calculate average gas price for each hour
average_gas_prices = process_blocks_and_calculate_average(start_block_number, end_block_number, step)

# Prepare data for the heatmap
heatmap_data = np.zeros((24, 7))  # 24 hours of the day, 7 days of the week
for hour, avg_gas_price in average_gas_prices.items():
    timestamp_datetime = datetime.datetime.utcfromtimestamp(hour * 3600)
    weekday = timestamp_datetime.weekday()  # Monday is 0 and Sunday is 6
    hour_of_day = timestamp_datetime.hour
    heatmap_data[hour_of_day][weekday] = avg_gas_price

# Plotting the heatmap using Seaborn with reversed colormap
days_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
hours_of_day = [str(i).zfill(2) + ':00' for i in range(24)]  # Zero-padded hours
sns.set(font_scale=1.2)
plt.figure(figsize=(14, 10))
ax = sns.heatmap(heatmap_data, cmap='hot_r', annot=True, fmt='.2f', cbar_kws={'label': 'Average gas price (Gwei)'})
ax.set_xticklabels(days_of_week)
ax.set_yticklabels(hours_of_day, rotation=0)
plt.title('Weekly Historic Heatmap of Average Gas Prices')
plt.xlabel('Days of the Week')
plt.ylabel('Hours of the Day')
plt.show()
